#使用Apriori算法进行关联分析

In [2]:
from numpy import *

def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [3]:
dataSet=loadDataSet()
print dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]


In [4]:
C1=createC1(dataSet)
print C1

[frozenset([1]), frozenset([2]), frozenset([3]), frozenset([4]), frozenset([5])]


In [5]:
D=map(set,dataSet)
print D

[set([1, 3, 4]), set([2, 3, 5]), set([1, 2, 3, 5]), set([2, 5])]


In [6]:
L1,suppData0=scanD(D,C1,0.5)
print L1

[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]


In [7]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [8]:
L,suppData=apriori(dataSet)
for each in L:
    print each

[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]
[frozenset([1, 3]), frozenset([2, 5]), frozenset([2, 3]), frozenset([3, 5])]
[frozenset([2, 3, 5])]
[]


In [9]:
aprioriGen(L[0],2)

[frozenset({1, 3}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({3, 5}),
 frozenset({2, 5})]

In [10]:
L,suppData=apriori(dataSet,minSupport=0.6)
for each in L:
    print each

[frozenset([3]), frozenset([2]), frozenset([5])]
[frozenset([2, 5])]
[]


In [11]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print freqSet-conseq,'-->',conseq,'conf:',conf
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [12]:
L,suppData=apriori(dataSet,minSupport=0.5)
rules=generateRules(L,suppData,minConf=0.7)
rules

frozenset([1]) --> frozenset([3]) conf: 1.0
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0


[(frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0)]

In [13]:
rules=generateRules(L,suppData,minConf=0.5)

frozenset([3]) --> frozenset([1]) conf: 0.666666666667
frozenset([1]) --> frozenset([3]) conf: 1.0
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0
frozenset([3]) --> frozenset([2]) conf: 0.666666666667
frozenset([2]) --> frozenset([3]) conf: 0.666666666667
frozenset([5]) --> frozenset([3]) conf: 0.666666666667
frozenset([3]) --> frozenset([5]) conf: 0.666666666667
frozenset([5]) --> frozenset([2, 3]) conf: 0.666666666667
frozenset([3]) --> frozenset([2, 5]) conf: 0.666666666667
frozenset([2]) --> frozenset([3, 5]) conf: 0.666666666667


In [14]:
#p220

In [16]:
mushDatSet=[line.split() for line in open('mushroom.dat').readlines() ]

In [17]:
L,suppData=apriori(mushDatSet,minSupport=0.3)
for item in L[1]:
    if item.intersection('2'):print item

frozenset(['2', '59'])
frozenset(['39', '2'])
frozenset(['2', '67'])
frozenset(['2', '34'])
frozenset(['2', '23'])
frozenset(['2', '86'])
frozenset(['76', '2'])
frozenset(['90', '2'])
frozenset(['2', '53'])
frozenset(['93', '2'])
frozenset(['63', '2'])
frozenset(['2', '28'])
frozenset(['2', '85'])
frozenset(['2', '36'])


In [18]:
L,suppData=apriori(mushDatSet,minSupport=0.3)
for item in L[3]:
    if item.intersection('2'):print item

frozenset(['63', '59', '2', '93'])
frozenset(['39', '2', '53', '34'])
frozenset(['2', '59', '23', '85'])
frozenset(['2', '59', '90', '85'])
frozenset(['39', '2', '36', '34'])
frozenset(['39', '63', '2', '85'])
frozenset(['39', '2', '90', '85'])
frozenset(['2', '59', '90', '86'])
frozenset(['2', '90', '93', '86'])
frozenset(['39', '2', '63', '86'])
frozenset(['39', '2', '93', '34'])
frozenset(['76', '39', '2', '34'])
frozenset(['39', '2', '59', '86'])
frozenset(['2', '59', '90', '36'])
frozenset(['2', '59', '93', '85'])
frozenset(['2', '59', '86', '34'])
frozenset(['2', '63', '90', '86'])
frozenset(['39', '63', '2', '36'])
frozenset(['2', '67', '86', '85'])
frozenset(['2', '28', '53', '85'])
frozenset(['2', '59', '93', '36'])
frozenset(['63', '23', '2', '34'])
frozenset(['2', '23', '36', '85'])
frozenset(['2', '90', '34', '85'])
frozenset(['39', '2', '59', '34'])
frozenset(['39', '2', '90', '34'])
frozenset(['2', '59', '28', '85'])
frozenset(['2', '34', '28', '85'])
frozenset(['2', '59'